In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, WhiteKernel
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (LSTM, Dense, Dropout, Bidirectional, Attention,
                                     Concatenate, Input, GlobalAveragePooling1D, GRU,
                                     Conv1D, LayerNormalization)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import warnings

warnings.filterwarnings('ignore')

# --- 1. Load and Clean Data (Adapted for Crude Oil) ---
print("1. Loading and cleaning crude oil price data...")
try:
    # Define column names for the crude oil data file
    column_names = ['Date', 'Crude_oil_average', 'Crude_oil_Brent', 'Crude_oil_Dubai', 'Crude_oil_WTI']
    
    # Read the Excel file for crude oil (Sheet1)
    data = pd.read_excel(
        'Average_oil_price.xlsx', sheet_name='Sheet1', skiprows=3, header=None,
        names=column_names, na_values=['…', '...', '–', '-', 'nan', '']
    )
    
    # Parse the date format (e.g., '1960M01')
    data['Date'] = pd.to_datetime(data['Date'].str.replace('M', '-'), errors='coerce')
    
    # Set the target column to 'Crude_oil_average'
    data['Crude_oil_average'] = pd.to_numeric(data['Crude_oil_average'], errors='coerce')
    data.dropna(subset=['Date', 'Crude_oil_average'], inplace=True)
    data = data.sort_values('Date').reset_index(drop=True)
    print(f"Data loaded successfully. Shape after cleaning: {data.shape}")
except FileNotFoundError:
    print("Error: 'Average_oil_price.xlsx' not found. Please ensure the file is in the correct directory.")
    exit()

# --- 2. Feature Engineering (with Cyclical Features) ---
print("\n2. Creating advanced features (including cyclical features)...")
df = data[['Date', 'Crude_oil_average']].copy()
df.set_index('Date', inplace=True)

# Basic time features
df['Year'] = df.index.year
month = df.index.month

# New: Convert month to cyclical features
df['Month_Sin'] = np.sin(2 * np.pi * month / 12)
df['Month_Cos'] = np.cos(2 * np.pi * month / 12)

# Trend features
df['Rolling_Avg_3M'] = df['Crude_oil_average'].rolling(window=3).mean()
df['Rolling_Avg_12M'] = df['Crude_oil_average'].rolling(window=12).mean()

df.bfill(inplace=True)
df.ffill(inplace=True)
print("New features created:")
print(df.head())

# --- 3. Data Preprocessing ---
print("\n3. Performing multivariate data preprocessing...")
# Update feature columns
feature_columns = ['Crude_oil_average', 'Year', 'Month_Sin', 'Month_Cos', 'Rolling_Avg_3M', 'Rolling_Avg_12M']
target_column = 'Crude_oil_average'

features = df[feature_columns].values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(features)

def create_multivariate_dataset(data, target_col_index, time_step=12):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i+time_step), :])
        y.append(data[i + time_step, target_col_index])
    return np.array(X), np.array(y)

time_step = 12
target_col_index = feature_columns.index(target_column)
X_full, y_full = create_multivariate_dataset(scaled_data, target_col_index, time_step)
num_features = X_full.shape[2]
print(f"Training data shape: X_full={X_full.shape}, y_full={y_full.shape}")

# --- 4. Build Tuned Model ---
print("\n4. Building the tuned deep learning model...")
def build_tuned_model(input_shape):
    input_layer = Input(shape=input_shape)
    conv_layer = Conv1D(80, kernel_size=3, activation='relu', padding='same')(input_layer)
    dropout_layer_0 = Dropout(0.3)(conv_layer)
    
    lstm_layer = Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer='l2'))(dropout_layer_0)
    dropout_layer_1 = Dropout(0.5)(lstm_layer)
    
    gru_layer = GRU(64, return_sequences=True, kernel_regularizer='l2')(dropout_layer_1)
    dropout_layer_2 = Dropout(0.5)(gru_layer)

    attention_layer = Attention()([dropout_layer_2, dropout_layer_2])
    attention_layer = Concatenate(axis=-1)([dropout_layer_2, attention_layer])
    
    norm_layer = LayerNormalization()(attention_layer)
    pooled_layer = GlobalAveragePooling1D()(norm_layer)
    
    dense_layer = Dense(80, activation='relu')(pooled_layer)
    output_layer = Dense(1)(dense_layer)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss='mean_squared_error')
    return model

# --- 5. Train Model ---
print("\n5. Training the tuned model...")
model = build_tuned_model(input_shape=(time_step, num_features))
model.summary()
early_stopping = EarlyStopping(monitor='loss', patience=20, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=10, min_lr=1e-7)
model.fit(X_full, y_full, epochs=200, batch_size=32, callbacks=[early_stopping, lr_scheduler], verbose=1)

# --- 6. Iterative Forecasting (with Dynamic Period Calculation) ---
print("\n6. Performing iterative forecasting...")
# Dynamically calculate the number of steps to forecast until Dec 2035
last_date = df.index[-1]
start_date = last_date + pd.DateOffset(months=1)
end_date = pd.to_datetime('2035-12-01')
num_forecast_steps = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month) + 1
print(f"Forecasting {num_forecast_steps} months from {start_date.strftime('%Y-%m')} to {end_date.strftime('%Y-%m')}.")

future_dates = pd.date_range(start=start_date, periods=num_forecast_steps, freq='MS')
last_sequence_scaled = scaled_data[-time_step:].tolist()
historical_prices = df[target_column].tolist()
future_predictions = []

for date in future_dates:
    print(f"Forecasting {date.strftime('%Y-%m')}...", end="\r")
    current_input = np.array([last_sequence_scaled])
    next_pred_scaled = model.predict(current_input, verbose=0)[0, 0]
    
    dummy_array = np.zeros((1, num_features))
    dummy_array[0, target_col_index] = next_pred_scaled
    next_pred_real = scaler.inverse_transform(dummy_array)[0, target_col_index]
    
    future_predictions.append(next_pred_real)
    historical_prices.append(next_pred_real)
    
    # Dynamically create features for the next time step, including cyclical ones
    next_year = date.year
    next_month = date.month
    next_month_sin = np.sin(2 * np.pi * next_month / 12)
    next_month_cos = np.cos(2 * np.pi * next_month / 12)
    next_roll_3m = pd.Series(historical_prices).rolling(window=3).mean().iloc[-1]
    next_roll_12m = pd.Series(historical_prices).rolling(window=12).mean().iloc[-1]
    
    next_features_real = np.array([[next_pred_real, next_year, next_month_sin, next_month_cos, next_roll_3m, next_roll_12m]])
    next_features_scaled = scaler.transform(next_features_real)[0]
    
    last_sequence_scaled.pop(0)
    last_sequence_scaled.append(next_features_scaled.tolist())

print("\nInitial forecast generated.")
initial_predictions = np.array(future_predictions)

# --- 7. Apply GPR for Smoothing and Confidence Intervals ---
print("\n7. Applying Gaussian Process Regression model...")
future_dates_numeric = np.array(range(len(future_dates))).reshape(-1, 1)
kernel = C(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2)) + WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e+1))
gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=1e-2, random_state=42)
gpr.fit(future_dates_numeric, initial_predictions)
gpr_predictions, gpr_std = gpr.predict(future_dates_numeric, return_std=True)
ci_upper = gpr_predictions + 1.96 * gpr_std
ci_lower = gpr_predictions - 1.96 * gpr_std
print("Gaussian Process Regression model processing complete.")

# --- 8. Save Final Results to CSV ---
print("\n8. Saving final forecast to CSV file...")
forecast_df = pd.DataFrame({
    'Date': future_dates,
    'Predicted_Price': gpr_predictions,
    'Confidence_Interval_Lower': ci_lower,
    'Confidence_Interval_Upper': ci_upper
})
forecast_df['Date'] = forecast_df['Date'].dt.strftime('%Y-%m')
output_filename = 'oil_price_forecast_to_2035.csv'
forecast_df.to_csv(output_filename, index=False)
print(f"\nForecast successfully saved to '{output_filename}'")
print("\n--- Forecast Sample ---")
print(forecast_df.head())



1. Loading and cleaning crude oil price data...
Data loaded successfully. Shape after cleaning: (779, 5)

2. Creating advanced features (including cyclical features)...
New features created:
            Crude_oil_average  Year  Month_Sin     Month_Cos  Rolling_Avg_3M  \
Date                                                                           
1960-01-01               1.63  1960   0.500000  8.660254e-01            1.63   
1960-02-01               1.63  1960   0.866025  5.000000e-01            1.63   
1960-03-01               1.63  1960   1.000000  6.123234e-17            1.63   
1960-04-01               1.63  1960   0.866025 -5.000000e-01            1.63   
1960-05-01               1.63  1960   0.500000 -8.660254e-01            1.63   

            Rolling_Avg_12M  
Date                         
1960-01-01             1.63  
1960-02-01             1.63  
1960-03-01             1.63  
1960-04-01             1.63  
1960-05-01             1.63  

3. Performing multivariate data prepr

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 12, 6)]              0         []                            
                                                                                                  
 conv1d_6 (Conv1D)           (None, 12, 80)               1520      ['input_7[0][0]']             
                                                                                                  
 dropout_14 (Dropout)        (None, 12, 80)               0         ['conv1d_6[0][0]']            
                                                                                                  
 bidirectional_6 (Bidirecti  (None, 12, 256)              214016    ['dropout_14[0][0]']          
 onal)                                                                                      

W0000 00:00:1759273719.016578       1 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" frequency: 2400 num_cores: 10 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 524288 l3_cache_size: 524288 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


24/24 [==============================] - 2s 19ms/step - loss: 4.5679 - lr: 5.0000e-04
Epoch 2/200
24/24 [==============================] - 1s 21ms/step - loss: 3.4865 - lr: 5.0000e-04
Epoch 3/200
24/24 [==============================] - 1s 22ms/step - loss: 2.7067 - lr: 5.0000e-04
Epoch 4/200
24/24 [==============================] - 1s 22ms/step - loss: 2.1221 - lr: 5.0000e-04
Epoch 5/200
24/24 [==============================] - 1s 24ms/step - loss: 1.6874 - lr: 5.0000e-04
Epoch 6/200
24/24 [==============================] - 0s 18ms/step - loss: 1.3642 - lr: 5.0000e-04
Epoch 7/200
24/24 [==============================] - 0s 17ms/step - loss: 1.1232 - lr: 5.0000e-04
Epoch 8/200
24/24 [==============================] - 0s 16ms/step - loss: 0.9420 - lr: 5.0000e-04
Epoch 9/200
24/24 [==============================] - 0s 17ms/step - loss: 0.8035 - lr: 5.0000e-04
Epoch 10/200
24/24 [==============================] - 0s 16ms/step - loss: 0.6970 - lr: 5.0000e-04
Epoch 11/200
24/24 [===========

W0000 00:00:1759273786.031938       1 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" frequency: 2400 num_cores: 10 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 524288 l3_cache_size: 524288 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


Forecasting 2035-12...
Initial forecast generated.

7. Applying Gaussian Process Regression model...
Gaussian Process Regression model processing complete.

8. Saving final forecast to CSV file...

Forecast successfully saved to '/Users/oushilin/Desktop/AAAI/Data/oil_price_forecast_to_2035.csv'

--- Forecast Sample ---
      Date  Predicted_Price  Confidence_Interval_Lower  \
0  2024-12        71.641331                  71.447615   
1  2025-01        71.922300                  71.753342   
2  2025-02        73.882032                  73.743624   
3  2025-03        77.037975                  76.899594   
4  2025-04        80.554433                  80.423773   

   Confidence_Interval_Upper  
0                  71.835047  
1                  72.091259  
2                  74.020440  
3                  77.176356  
4                  80.685092  
